In [41]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os

In [42]:
shp = gpd.read_file(r"..\..\Data\Interim\PT-FireSprd_v2.1\L2_FireBehavior\PT-FireProg_v2.1_L2_p_meteo.shp")

print(f"Total de colunas: {len(shp.columns)}")
for i, coluna in enumerate(shp.columns, 1):
    print(f"{i:3d}. {coluna}")

Total de colunas: 116
  1. fid
  2. fname
  3. year
  4. id
  5. type
  6. sdate
  7. edate
  8. inidoy
  9. enddoy
 10. source
 11. zp_link
 12. burn_perio
 13. area
 14. growth_rat
 15. ros_i
 16. ros_p
 17. spdir_i
 18. spdir_p
 19. int_i
 20. int_p
 21. duration_i
 22. duration_p
 23. qc
 24. 1_3y_fir_p
 25. 3_8y_fir_p
 26. 8_ny_fir_p
 27. elev_av
 28. aspect_av
 29. landform
 30. fuel_model
 31. f_load_av
 32. land_use
 33. land_use_d
 34. CBH_m_av
 35. HigCC_p_av
 36. LowCC_p_av
 37. MidCC_p_av
 38. TotCC_p_av
 39. BLH_m_av
 40. Cape_av
 41. Cin_av
 42. gp_m2s2_av
 43. t_2m_C_av
 44. d_2m_C_av
 45. sP_hPa_av
 46. wv10_kh_av
 47. wdir10_av
 48. wv100_k_av
 49. wdir100_av
 50. rh_2m_av
 51. VPD_Pa_av
 52. dfmc_av
 53. sW_1m_av
 54. sW_3m_av
 55. LCL_hPa_av
 56. LCL_m_av
 57. HDW_av
 58. Haines_av
 59. wSv_9_av
 60. wSdir_9_av
 61. wSv_7_av
 62. wSdir_7_av
 63. wSv_5_av
 64. wSdir_5_av
 65. wSv_1_av
 66. wSdir_1_av
 67. gT_s_9_av
 68. gT_9_8_av
 69. gT_8_7_av
 70. gT_7_5_av
 71. gT_

In [43]:
columns_to_lag = ["ros_p"]
n_lags = 10
shp_combined = shp.sort_values(["fname", "zp_link", "sdate"]).reset_index(drop=True)

# garantir datetime
shp_combined["sdate"] = pd.to_datetime(shp_combined["sdate"], errors='coerce')
shp_combined["edate"] = pd.to_datetime(shp_combined["edate"], errors='coerce')

for col in columns_to_lag:
    if col not in shp_combined.columns:
        print(f"⚠️ Coluna '{col}' não encontrada. Pulando.")
        continue

    for lag in range(1, n_lags + 1):
        lag_col_name = f"{col}_lg{lag}"
        shp_combined[lag_col_name] = np.nan

        # processa por grupo (fname, zplink)
        for (fname, zplink), group in shp_combined.groupby(["fname", "zp_link"]):
            group = group.sort_values("sdate")
            group_idx = group.index

            for idx in group_idx:
                current_time = shp_combined.loc[idx, "sdate"]  # início do evento atual
                target_time = current_time - pd.Timedelta(hours=lag)

                # procurar a feature anterior que estava ativa no instante target_time
                mask = (group["sdate"] <= target_time) & (group["edate"] > target_time)

                if mask.any():
                    active_row = group.loc[mask].iloc[-1]
                    shp_combined.at[idx, lag_col_name] = active_row[col]
                else:
                    shp_combined.at[idx, lag_col_name] = np.nan

print("✅ Lags temporais adicionados com base na feature ativa há X horas.")

shp_lags = shp_combined

C:\Users\dias3\AppData\Local\Temp\ipykernel_9280\373017451.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  shp_combined["sdate"] = pd.to_datetime(shp_combined["sdate"], errors='coerce')


✅ Lags temporais adicionados com base na feature ativa há X horas.


In [ ]:
def calculate_fire_durations(df):
    """
    Calculates 'duration' (time since fire start) and lag features 
    (time differences between consecutive observations).
    """
    df_temp = df.copy()
    
    # 1. Convert sdate to datetime
    df_temp['sdate'] = pd.to_datetime(df_temp['sdate'], errors='coerce')
    
    # 2. Sort by fire name and date
    df_temp = df_temp.sort_values(by=['fname', 'sdate'])
    
    # 3. Calculate 'duration' (time since the start of the fire)
    fire_start_times = df_temp.groupby('fname')['sdate'].transform('min')
    df_temp['f_start'] = (df_temp['sdate'] - fire_start_times).dt.total_seconds() / 60
    
    
    return df_temp

# Apply the corrected function
shp_processed = calculate_fire_durations(shp_lags)

# --- Verification ---
print("\nVerifying calculation for 'Agueda_08082016':")
print(shp_processed[shp_processed['fname'] == 'Agueda_08082016'][
    ['fname', 'sdate', 'duration']
].head(10))

print("\nVerifying calculation for 'Gouveia_10082015':")
print(shp_processed[shp_processed['fname'] == 'Gouveia_10082015'][
    ['fname', 'sdate', 'duration']
].head(10))




Verifying calculation for 'Agueda_08082016':
              fname               sdate  duration
70  Agueda_08082016 2016-08-08 04:00:00       0.0
71  Agueda_08082016 2016-08-08 04:00:00       0.0
72  Agueda_08082016 2016-08-08 12:30:00     510.0
73  Agueda_08082016 2016-08-08 12:30:00     510.0
74  Agueda_08082016 2016-08-10 13:30:00    3450.0
75  Agueda_08082016 2016-08-10 13:30:00    3450.0
76  Agueda_08082016 2016-08-10 13:30:00    3450.0
49  Agueda_08082016                 NaT       NaN
50  Agueda_08082016                 NaT       NaN
51  Agueda_08082016                 NaT       NaN

Verifying calculation for 'Gouveia_10082015':
                 fname               sdate  duration
1558  Gouveia_10082015 2015-08-10 14:30:00       0.0
1559  Gouveia_10082015 2015-08-10 14:30:00       0.0
1560  Gouveia_10082015 2015-08-11 03:00:00     750.0
1561  Gouveia_10082015 2015-08-11 12:30:00    1320.0
1562  Gouveia_10082015 2015-08-11 12:30:00    1320.0
1563  Gouveia_10082015 2015-08-11 12:30

In [45]:
"""
Create fire front ranking system:
    -3: Other missing values
    -2: No meteorological data because of short time interval
    -1: Missing sdate or edate
    0: Single fire front for specific fire at specific time
    1: Multiple fire fronts - highest ROS_p
    2: Multiple fire fronts - lowest ROS_p
"""
shp_processed['fire_rank'] = 0

# Handle missing dates
missing_mask = shp_processed['sdate'].isna() | shp_processed['edate'].isna()
shp_processed.loc[missing_mask, 'fire_rank'] = -1


# Handle intervals that don't contain exact hour (-2)
def interval_has_full_hour(row):
    if pd.isna(row['sdate']) or pd.isna(row['edate']):
        return False
    # Generate range of hours
    s = row['sdate'].ceil('H')  # next full hour
    e = row['edate'].floor('H') # last full hour
    return s <= e

mask_no_full_hour = (~missing_mask) & (~shp_processed.apply(interval_has_full_hour, axis=1))
shp_processed.loc[mask_no_full_hour, 'fire_rank'] = -2

# Handle missing meteorological data (-3)
mask_missing_meteo = shp_processed['t_2m_C_av'].isna() & (shp_processed['ros_p'] != -1) & (shp_processed['fire_rank'] != -2)
shp_processed.loc[mask_missing_meteo, 'fire_rank'] = -3

# Process valid rows
valid_mask = (~missing_mask) & (~mask_no_full_hour)
for (fname, sdate), group in shp_processed[valid_mask].groupby(['fname', 'sdate']):
    valid_rows = group[(group['type'] == 'p') & (group['ros_p'] > 0) & (group['fire_rank'] == 0)]
    if len(valid_rows) > 1:
        max_ros_idx = valid_rows['ros_p'].idxmax()
        min_ros_idx = valid_rows['ros_p'].idxmin()
        shp_processed.loc[max_ros_idx, 'fire_rank'] = 1
        shp_processed.loc[min_ros_idx, 'fire_rank'] = 2

# Display results
rank_counts = shp_processed['fire_rank'].value_counts().sort_index()
print("Fire front rank distribution:")
print(rank_counts)
print(f"\nTotal rows: {len(shp_processed)}")

C:\Users\dias3\AppData\Local\Temp\ipykernel_9280\2298485294.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  s = row['sdate'].ceil('H')  # next full hour
C:\Users\dias3\AppData\Local\Temp\ipykernel_9280\2298485294.py:23: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  e = row['edate'].floor('H') # last full hour


Fire front rank distribution:
fire_rank
-3       3
-2      14
-1    1640
 0    1020
 1     339
 2     339
Name: count, dtype: int64

Total rows: 3355


In [46]:
for i, coluna in enumerate(shp_processed.columns, 1):
    print(f"{i:3d}. {coluna}")

  1. fid
  2. fname
  3. year
  4. id
  5. type
  6. sdate
  7. edate
  8. inidoy
  9. enddoy
 10. source
 11. zp_link
 12. burn_perio
 13. area
 14. growth_rat
 15. ros_i
 16. ros_p
 17. spdir_i
 18. spdir_p
 19. int_i
 20. int_p
 21. duration_i
 22. duration_p
 23. qc
 24. 1_3y_fir_p
 25. 3_8y_fir_p
 26. 8_ny_fir_p
 27. elev_av
 28. aspect_av
 29. landform
 30. fuel_model
 31. f_load_av
 32. land_use
 33. land_use_d
 34. CBH_m_av
 35. HigCC_p_av
 36. LowCC_p_av
 37. MidCC_p_av
 38. TotCC_p_av
 39. BLH_m_av
 40. Cape_av
 41. Cin_av
 42. gp_m2s2_av
 43. t_2m_C_av
 44. d_2m_C_av
 45. sP_hPa_av
 46. wv10_kh_av
 47. wdir10_av
 48. wv100_k_av
 49. wdir100_av
 50. rh_2m_av
 51. VPD_Pa_av
 52. dfmc_av
 53. sW_1m_av
 54. sW_3m_av
 55. LCL_hPa_av
 56. LCL_m_av
 57. HDW_av
 58. Haines_av
 59. wSv_9_av
 60. wSdir_9_av
 61. wSv_7_av
 62. wSdir_7_av
 63. wSv_5_av
 64. wSdir_5_av
 65. wSv_1_av
 66. wSdir_1_av
 67. gT_s_9_av
 68. gT_9_8_av
 69. gT_8_7_av
 70. gT_7_5_av
 71. gT_5_3_av
 72. CMLG_av
 7

In [ ]:
shp_processed['HigCC_p_av'] = shp_processed['HigCC_p_av'] * 100
shp_processed['MidCC_p_av'] = shp_processed['MidCC_p_av'] * 100
shp_processed['LowCC_p_av'] = shp_processed['LowCC_p_av'] * 100
shp_processed['TotCC_p_av'] = shp_processed['TotCC_p_av'] * 100

shp_processed['sW_1m_av'] = shp_processed['sW_1m_av'] * 100
shp_processed['sW_3m_av'] = shp_processed['sW_3m_av'] * 100
shp_processed['sW_7_av'] = shp_processed['sW_7_av'] * 100
shp_processed['sW_28_av'] = shp_processed['sW_28_av'] * 100
shp_processed['sW_100_av'] = shp_processed['sW_100_av'] * 100
shp_processed['sW_289_av'] = shp_processed['sW_289_av'] * 100

In [47]:
shp_processed['sdate'] = shp_processed['sdate'].astype(str)
shp_processed['edate'] = shp_processed['edate'].astype(str)

In [48]:
os.makedirs(r'..\..\Data\Processed\PT-FireSprd_v2.1\L2_FireBehavior', exist_ok=True)
shp_processed.to_file(r'..\..\Data\Processed\PT-FireSprd_v2.1\L2_FireBehavior\PT-FireProg_v2.1_L2_final.shp')